In [ ]:
!pip3 install catboost
!pip install fancyimpute
!pip install category_encoders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
cleve_df =  pd.read_csv("/content/drive/MyDrive/wiecon paper/dataset/processed.cleveland.data", names=columns)
print("Number of rows: %d" % cleve_df.shape[0])
print("Number of columns: %d" % cleve_df.shape[1])
hung_df =  pd.read_csv("/content/drive/MyDrive/wiecon paper/dataset/processed.hungarian.data",names=columns )
print("Number of rows: %d" % hung_df.shape[0])
print("Number of columns: %d" % hung_df.shape[1])
switz_df =  pd.read_csv("/content/drive/MyDrive/wiecon paper/dataset/processed.switzerland.data", names=columns)
print("Number of rows: %d" % switz_df.shape[0])
print("Number of columns: %d" % switz_df.shape[1])
va_df =  pd.read_csv("/content/drive/MyDrive/wiecon paper/dataset/processed.va.data", names=columns)
print("Number of rows: %d" % va_df.shape[0])
print("Number of columns: %d" % va_df.shape[1])
statlog_df = pd.read_csv('/content/drive/MyDrive/wiecon paper/dataset/statlog_space_separated.data', sep='\\s+', header=None, names=columns)
print("Number of rows: %d" % statlog_df.shape[0])
print("Number of columns: %d" % statlog_df.shape[1])


Number of rows: 303
Number of columns: 14
Number of rows: 294
Number of columns: 14
Number of rows: 123
Number of columns: 14
Number of rows: 200
Number of columns: 14
Number of rows: 270
Number of columns: 14


In [ ]:
# 1 means No heart disease in this dataset
statlog_df['num'] = statlog_df['num'].replace(1, 0)

In [ ]:
statlog_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,2
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,0
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,2
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,0
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,0


In [ ]:
df = pd.concat([cleve_df, hung_df, switz_df, va_df, statlog_df])
print("Total number of rows: %d" % df.shape[0])
print("Number of columns: %d" % df.shape[1])

Total number of rows: 1190
Number of columns: 14


In [ ]:
# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

In [ ]:
# Replace the old value with the new value in column 1
df['num'] = df['num'].replace(2, 1)
df['num'] = df['num'].replace(3, 1)
df['num'] = df['num'].replace(4, 1)

In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plot_distplot(df):
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Create a distplot for the current column
        sns.displot(df[column], kde=True)

        # Set plot title and labels
        plt.title(f"Distribution of {column}")
        plt.xlabel(column)
        plt.ylabel("Density")

        # Show the plot
        plt.show()

# Example usage
# Assuming you have a dataframe called 'data' with multiple columns

#plot_distplot(df)


In [ ]:
## Converting Zero Value of Cholesterol to NaN Value
df.loc[df['chol'] == 0,'chol'] = np.nan

# df["chol"] = df["chol"].fillna(df["chol"].median())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# create the MinMaxScaler object
scaler = MinMaxScaler()

# fit and transform the DataFrame
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
missing_values = df_normalized.isnull().sum()

# Print the result
print(missing_values)

age           0
sex           0
cp            0
trestbps     59
chol        153
fbs          90
restecg       2
thalach      55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64


In [ ]:
from sklearn.impute import KNNImputer

# Create a copy of the DataFrame
df_filled = df_normalized.copy()

# Select the 'oldpeak' column
oldpeak_data = df_filled[['oldpeak']]

# Initialize the KNN imputer
imputer = KNNImputer(n_neighbors=7)

# Fill the missing values in the 'oldpeak' column
oldpeak_data_filled = imputer.fit_transform(oldpeak_data)

# Assign the filled values back to the 'oldpeak' column
df_normalized['oldpeak'] = oldpeak_data_filled

In [ ]:
import numpy as np
import pandas as pd

# Assuming df_normalized is your DataFrame

# Step 1: Fill missing values with mean of respective columns
df_filled = df_normalized.fillna(df_normalized.mean())

# Step 2: Convert the DataFrame to a numpy array
data_array = df_filled.values

# Step 3: Perform Singular Value Decomposition (SVD)
U, sigma, Vt = np.linalg.svd(data_array, full_matrices=False)

# Step 4: Reconstruct the matrix using SVD components, considering only significant singular values
rank = np.linalg.matrix_rank(data_array)
reconstructed_matrix = np.dot(U[:, :rank] * sigma[:rank], Vt[:rank, :])

# Step 5: Replace missing values in the original DataFrame with values from the reconstructed matrix
df_filled.iloc[:, :] = reconstructed_matrix

# Step 6: Check if there are any missing values left
print(df_filled.isnull().sum())


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64


In [ ]:
missing_values = df_normalized.isnull().sum()

# Print the result
print(missing_values)

age           0
sex           0
cp            0
trestbps     59
chol        153
fbs          90
restecg       2
thalach      55
exang        55
oldpeak       0
slope       309
ca          611
thal        486
num           0
dtype: int64


In [ ]:
from fancyimpute import IterativeImputer
# convert DataFrame to numpy array
data = df_normalized.to_numpy()
#data = df.to_numpy()
# create an instance of the IterativeImputer class
imputer = IterativeImputer()

# impute missing values using MICE
data_imputed = imputer.fit_transform(data)

# convert numpy array back to DataFrame
df_imputed = pd.DataFrame(data_imputed, columns=df_normalized.columns)



In [ ]:
missing_values = df_imputed.isnull().sum()

# Print the result
print(missing_values)

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix, accuracy_score , recall_score , precision_score

# separate the target variable from the features
X = df_imputed.drop('num', axis=1)
y = df_imputed['num']


In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X.to_numpy())
mask=yhat!=-1
X_en=X.to_numpy()[mask, :]
X=pd.DataFrame(X_en, columns=X.columns)
y=y[mask]


In [ ]:
X.shape,y.shape

((1120, 13), (1120,))

In [ ]:
# Splitting data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further splitting training data into training and validation sets (60% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
# Print the shapes of the resulting sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (806, 13) (806,)
Validation set shape: (90, 13) (90,)
Testing set shape: (224, 13) (224,)
